### SRE strategy

In [3]:
import pandas as pd
import numpy as np


In [3]:
# fut = pd.read_pickle("C:\\PICKLE\\Nifty Future\\2019-02-11_nifty_future.pkl").set_index("date_time")
# fut.index = pd.to_datetime(fut.index)
# opt = pd.read_pickle("C:\\PICKLE\\Nifty Options\\2019-02-11_nifty.pkl").set_index("date_time")
# opt.index = pd.to_datetime(opt.index)

In [53]:
fut = pd.read_pickle("C:\\PICKLE\\BN Future\\2025-08-04_banknifty_future.pkl").set_index("date_time")
fut.index = pd.to_datetime(fut.index)
opt = pd.read_pickle("C:\\PICKLE\\BN Options\\2025-08-04_banknifty.pkl").set_index("date_time")
opt.index = pd.to_datetime(opt.index)

In [54]:
entry_time = pd.to_datetime("2025-08-04 09:20:00")
exit_time = pd.to_datetime("2025-08-04 15:25:00")
sl =  10
intra_sl = 20
om = 0.1

In [55]:
# import pandas as pd
def get_one_om(future_price,fpt,step):
    future_price = fpt['close'].iloc[0] if future_price is None else future_price

    return ((int(future_price/step)*step)/100)

def get_strangle_strike(start_dt,end_dt,opt:pd.DataFrame,fpt:pd.DataFrame,step:int,om=0,target = False,tf = 1):

    valid_times = fpt.loc[start_dt:end_dt].index
    for current_dt in valid_times:
        # om , target ,tf= 1 , 1,1
        future_price = fpt.loc[current_dt,'close']
        one_om = get_one_om(future_price,fpt,step)
        target = one_om * om if target is None else target
        # opt[(opt.index==current_dt)]
        target_od = opt[(opt.index==current_dt)&(opt['close']>=target*tf)].sort_values(by=['close']).copy()

        ce_scrip = target_od.loc[target_od['scrip'].str.endswith('CE'),'scrip'].iloc[0]
        pe_scrip = target_od.loc[target_od['scrip'].str.endswith('PE'),'scrip'].iloc[0]
        ce_scrip_list = [ce_scrip,f"{int(ce_scrip[:-2])-step}CE",f"{int(ce_scrip[:-2])+step}CE"]
        pe_scrip_list = [pe_scrip,f"{int(pe_scrip[:-2])-step}PE",f"{int(pe_scrip[:-2])+step}PE"]

        call_list_prices, put_list_prices = [], []
        for z in range(3):
            try:
                call_list_prices.append(opt[opt['scrip'] == ce_scrip_list[z]].loc[current_dt, 'close'])
            except:
                call_list_prices.append(0)
            try:
                put_list_prices.append(opt[opt['scrip'] == pe_scrip_list[z]].loc[current_dt, 'close'])
            except:
                put_list_prices.append(0)
        call,put,min_diff = call_list_prices[0],put_list_prices[0],float('inf')
        target_2, target_3 = target*2*tf, target*3

        diff = abs(put-call)
        required_call, required_put = None, None
        if (put+call >= target_2) & (min_diff > diff) & (put+call <= target_3):
            min_diff = diff
            required_call, required_put = call, put

        for i in range(1,3):
            if (min_diff > abs(put_list_prices[i] - call)) & (put_list_prices[i]+call >= target_2) & (put_list_prices[i]+call <= target_3):
                min_diff = abs(put_list_prices[i] - call)
                required_call, required_put = call, put_list_prices[i]
            if (min_diff > abs(call_list_prices[i] - put)) & (call_list_prices[i]+put >= target_2) & (call_list_prices[i]+put <= target_3):
                min_diff = abs(call_list_prices[i] - put)
                required_call, required_put = call_list_prices[i], put

        ce_scrip, pe_scrip = ce_scrip_list[call_list_prices.index(required_call)], pe_scrip_list[put_list_prices.index(required_put)]
        ce_price, pe_price = opt[opt['scrip'] == ce_scrip].loc[current_dt, 'close'], opt[opt['scrip'] == pe_scrip].loc[current_dt, 'close']
            

        return ce_scrip,pe_scrip,ce_price,pe_price
       

In [56]:
ce_scrip,pe_scrip,ce_price,pe_price = get_strangle_strike(entry_time,exit_time,opt,fut,100,om=om,target = None,tf = 1)
ce_price,pe_price,ce_scrip,pe_scrip

(np.float64(59.3), np.float64(59.2), '58400CE', '52700PE')

In [57]:
opt = opt.loc[entry_time:exit_time]
ce_data = opt[(opt['scrip'] == ce_scrip) ]
pe_data = opt[(opt['scrip'] == pe_scrip) ]

In [58]:
sl =  8
intra_sl = 20
ce_pe_price = ce_price+pe_price
sl_price = ce_pe_price*(1+sl/100)
intra_sl_price = ce_pe_price*(1+intra_sl/100)
sl_price,ce_pe_price ,intra_sl_price

(np.float64(127.98), np.float64(118.5), np.float64(142.2))

In [72]:
def check_sl_hit(ce_data,pe_data,sl_price,intra_sl_price):
    try:
        sl_time = ce_data[ce_data.close +  pe_data.close >= sl_price].index[0]
    except:
        sl_time = None
    try:
        intra_sl_time = ce_data[np.maximum(ce_data.high + pe_data.low, ce_data.low + pe_data.high )>= intra_sl_price ].index[0]
        
    except:
        intra_sl_time = None

    return sl_time,intra_sl_time


In [ ]:
def calculate_sl_intra(entry_time):
    ce_pe_price = ce_data.loc[entry_time,'close']+pe_data.loc[entry_time,'close']
    sl =  8
    intra_sl = 20
    sl_price = ce_pe_price*(1+sl/100)
    intra_sl_price = ce_pe_price*(1+intra_sl/100)
    
    return sl_price,ce_pe_price ,intra_sl_price


In [61]:
sl_time,intra_sl_time = check_sl_hit(ce_data,pe_data,sl_price,intra_sl_price)
sl_time,intra_sl_time

(Timestamp('2025-08-04 09:48:00'), None)

In [ ]:
total_pnl=0
sl_time,intra_sl_time = check_sl_hit(ce_data,pe_data,sl_price,intra_sl_price)
while True:

    if sl_time is None and intra_sl_time is None:
        print("SL NOT HIT", total_pnl )
        exit_ce_pe = ce_data.loc[exit_time,'close'] + pe_data.loc[exit_time,'close']
        pnl = ce_pe_price - exit_ce_pe
        print("pnl at exit", pnl)
        total_pnl += pnl
        print("total pnl ",total_pnl) 
        break

    elif(sl_time is not None and intra_sl_time is not None):
        if sl_time < intra_sl_time:
            print("SL HIT : ", sl_time)
            pnl = ce_pe_price - sl_price
            total_pnl +=pnl
            print("pnl ",pnl)
            print('total pnl ',total_pnl)
            entry_time = sl_time            
            sl_price,ce_pe_price ,intra_sl_price = calculate_sl_intra(entry_time)

            sl_time,intra_sl_time = check_sl_hit(ce_data,pe_data,sl_price,intra_sl_price)
            print("next trade hit")
        else:
            print("INTRA SL HIT ",intra_sl)
            pnl = ce_pe_price - intra_sl_price
            print(pnl)
            break
    elif sl_time is not None and intra_sl_time is None:
        
        print("SL HIT : ", sl_time)
        pnl = ce_pe_price - sl_price
        total_pnl +=pnl
        print("pnl ",pnl)
        print('total pnl ',total_pnl)
        entry_time = sl_time            
        sl_price,ce_pe_price ,intra_sl_price = calculate_sl_intra(entry_time)
        sl_time,intra_sl_time = check_sl_hit(ce_data,pe_data,sl_price,intra_sl_price)
        print("next trade hit")
         

SL HIT :  2025-08-04 09:48:00
pnl  -9.480000000000004
total pnl  -9.480000000000004
next trade hit
SL NOT HIT -9.480000000000004
pnl at exit 22.849999999999994
total pnl  13.36999999999999


In [ ]:
pd.DataFrame(np.maximum(ce_data.high + pe_data.low, ce_data.low + pe_data.high ) )

In [63]:
def calculate_sl_intra(entry_time,sl,intra_sl):
    ce_pe_price = ce_data.loc[entry_time,'close']+pe_data.loc[entry_time,'close']

    sl_price = ce_pe_price*(1+sl/100)
    intra_sl_price = ce_pe_price*(1+intra_sl/100)
    return sl_price,ce_pe_price ,intra_sl_price


In [67]:
def backtest_strategy(opt, fut, entry_time, exit_time, ce_scrip, pe_scrip, sl, intra_sl):
    
    fut = pd.read_pickle("C:\\PICKLE\\BN Future\\2025-08-04_banknifty_future.pkl").set_index("date_time")
    fut.index = pd.to_datetime(fut.index)
    opt = pd.read_pickle("C:\\PICKLE\\BN Options\\2025-08-04_banknifty.pkl").set_index("date_time")
    opt.index = pd.to_datetime(opt.index)

    ce_scrip,pe_scrip,ce_price,pe_price = get_strangle_strike(entry_time,exit_time,opt,fut,100,om=om,target = None,tf = 1)

    opt = opt.loc[entry_time:exit_time]
    ce_data = opt[(opt['scrip'] == ce_scrip) ]
    pe_data = opt[(opt['scrip'] == pe_scrip) ]
  
    sl_price,ce_pe_price ,intra_sl_price = calculate_sl_intra(entry_time,sl,intra_sl)
    # print("price sl , ce_pe",sl_price,ce_pe_price)
    
    total_pnl=0
    
    sl_time,intra_sl_time = check_sl_hit(ce_data,pe_data,sl_price,intra_sl_price)
    # print("sl_hit" , sl_time,intra_sl_time)
    next_hit = False
    while True:
        if sl_time is None and intra_sl_time is None:
            print("SL NOT HIT", total_pnl )
            exit_ce_pe = ce_data.loc[exit_time,'close'] + pe_data.loc[exit_time,'close']
            pnl = ce_pe_price - exit_ce_pe
            print("pnl at exit", pnl)
            total_pnl += pnl
            print("total pnl ",total_pnl) 
            break

        elif(sl_time is not None and intra_sl_time is not None):
            if sl_time < intra_sl_time:
                print("SL HIT : ", sl_time)
                pnl = ce_pe_price - sl_price
                total_pnl +=pnl
                print("pnl ",pnl)
                print('total pnl ',total_pnl)
                
                sl_price,ce_pe_price ,intra_sl_price = calculate_sl_intra(sl_time,sl,intra_sl)

                sl_time,intra_sl_time = check_sl_hit(ce_data,pe_data,sl_price,intra_sl_price)
                next_hit = False
                print("next trade hit")
            else:
                print("INTRA SL HIT ",intra_sl)
                pnl = ce_pe_price - intra_sl_price
                print(pnl)
                break
        elif sl_time is not None and intra_sl_time is None:
            
            print("SL HIT : ", sl_time)
            pnl = ce_pe_price - sl_price
            total_pnl +=pnl
            print("pnl ",pnl)
            print('total pnl ',total_pnl)
            sl_price,ce_pe_price ,intra_sl_price = calculate_sl_intra(sl_time,sl,intra_sl)
            sl_time,intra_sl_time = check_sl_hit(ce_data,pe_data,sl_price,intra_sl_price)
            next_hit = False
            print("next trade hit")
            

    return {"total_pnl" : total_pnl,
            "next trade" : next_hit
    }

    

In [71]:
backtest_strategy(opt, fut,entry_time, exit_time, ce_scrip, pe_scrip, sl, intra_sl)

SL HIT :  2025-08-04 09:48:00
pnl  -9.480000000000004
total pnl  -9.480000000000004
next trade hit
SL NOT HIT -9.480000000000004
pnl at exit 22.849999999999994
total pnl  13.36999999999999


{'total_pnl': np.float64(13.36999999999999), 'next trade': False}